In [52]:
#!/usr/bin/env python
""" 
"""
__author__ = 'Adam Duster'
__copyright__ = ''
__credits__ = ['Adam Duster']
__license__ = 'CC-BY-SA'
__version__ = '0.1'
__email__ = 'adam.duster@ucdenver.edu'
__status__ = 'Development'

import argparse
def get_args(args=None):
    """ This is written as a default funtion to put at beginning of all Python
    scripts which require command line arguments. This uses the argparse module
    which must be declared in the main program to ensure that the object is able
    to be used by the caller
    --Adam Duster 21 June 2017
    """
    parser = argparse.ArgumentParser(description='see header of python script')
    parser.add_argument(
        '-i',
        '--if_paths',
        help='The first and second files in order',
        required=True)
    parser.add_argument(
        '-v',
        '--verbose',
        help='Controls the level of output, use multipe v for more output',
        required=False,
        action='count',
        default=0)
    parser.add_argument(
        '-s',
        '--step',
        help='The change in coordinates between the two files',
        required=False,
        default=1e-6,
        type=float
    )
    parser.add_argument(
        '-d',
        '--debug',
        help='Enter debug mode',
        required=False,
        action='store_true',
        default=False)
    return parser.parse_args(args)

## Vars
arg_vals = None
args = get_args(['-i', 'oh-bpsf.h5', '-s', '1e-6'])



In [53]:
# Calculate the numerical gradient
import h5py as h5
import numpy as np

#key = 'o_rad_cartesian_gradient'
#key = 'h_radial_sym_funcs'
key = 'o_radial_sym_funcs'
ckey = 'o_rad_cartesian_gradient'
check = False
check = True
print(args.if_paths[0])
print(args.if_paths[1])
k = 0
with h5.File(args.if_paths, 'r') as ifi:
    delta_symfuncs = (ifi[key][1,:] - ifi[key][0,:] ) / 1e-6
    print(ckey, 'shape: ', ifi[ckey].shape)
    ograd = ifi[ckey][1]
    hgrad = ifi['h_rad_cartesian_gradient'][1]
    o_symfuncs = ifi['o_radial_sym_funcs'][1]
    h_symfuncs = ifi['h_radial_sym_funcs'][1]
#print(calc)

o
h
o_rad_cartesian_gradient shape:  (2, 48, 151, 3)


In [54]:
for i in range(delta_symfuncs.size):
    if delta_symfuncs[i] == 0 and calc[i] == 0:
        continue
    print(delta_symfuncs[i], calc[i], calc[i]-delta_symfuncs[i], (calc[i]-delta_symfuncs[i])/delta_symfuncs[i])

-3.4570693969726562 [ 3.47510004 -3.47510004] [ 6.93216944 -0.01803064] [-2.00521559  0.00521559]
1.8477439880371094 [-1.86287606  1.86287606] [-3.71062005  0.01513207] [-2.00818948  0.00818948]
1.6391277313232422 [-1.62611103  1.62611103] [-3.26523876 -0.0130167 ] [-1.99205876 -0.00794124]
0.6109476089477539 [-0.60673434  0.60673434] [-1.21768194 -0.00421327] [-1.99310371 -0.00689629]
0.22351741790771484 [-0.22193363  0.22193363] [-0.44545105 -0.00158378] [-1.99291427 -0.00708573]
0.08940696716308594 [-0.09001059  0.09001059] [-0.17941756  0.00060362] [-2.00675142  0.00675142]
0.040978193283081055 [-0.04100074  0.04100074] [-8.19789320e-02  2.25454569e-05] [-2.00055018e+00  5.50181818e-04]
0.020954757928848267 [-0.02073114  0.02073114] [-0.04168589 -0.00022362] [-1.98932836 -0.01067164]
0.011408701539039612 [-0.01145233  0.01145233] [-2.28610281e-02  4.36250120e-05] [-2.00382384  0.00382384]
0.006868503987789154 [-0.0068111  0.0068111] [-1.36796082e-02 -5.73997386e-05] [-1.99164305 -0

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [86]:
npa = np.asarray


xi = npa([0.,0.,0.])
xj = npa([1.,0.,0.])
def calc_gi(xi, xj, rs=0.800, eta=19.531, rc=8.0, verbose=False):
    vij = xj - xi
    rij = np.linalg.norm(vij)
    drijdi = -vij/rij
    drijdj = drijdi
    fc = 0.5 * (np.cos(np.pi*rij/rc) + 1)
    gauss = np.exp(-eta*(rij - rs)**2)
    gi = gauss*fc
    if verbose:
        print("rij, then rs", rij, rs)
        print("(rij-rs)**2",(rij-rs)**2)
        print("myexp", gauss)
        print("myfc", fc)
    
    return gi

cgi = calc_gi(xi, xj)
print("calc gi", cgi)
print("reference_gi", o_symfuncs[0])


calc gi 0.44041252110981277
reference_gi 0.4404081106185913


In [80]:
# Lets calculate the O symfunc numerical gradient
s = 1e-6
xi = npa([0.,0.,0.])
xj = npa([1.,0.,0.])
xib = np.copy(xi)
xib = xib[0] + s

agi = calc_gi(xi, xj)
bgi = calc_gi(xib, xj)
print("numerical gradient: ", (bgi - agi)/s)
print(ograd[0,0])


rij, then rs 1.0 0.8
(rij-rs)**2 0.03999999999999998
myexp 0.4578379401281239
myfc 0.9619397662556434
rij, then rs 0.999999000001 0.8
(rij-rs)**2 0.03999960000139996
myexp 0.45784151694270037
myfc 0.9619398413952489
numerical gradient:  3.475082208603819
[3.47508979 0.         0.        ]


In [89]:
# Lets calculate the H symfunc numerical gradient
s = 1e-6
xi = npa([1.,0.,0.])
xj = npa([0.,0.,0.])
xib = np.copy(xi)
xib[0] = xib[0] + s

agi = calc_gi(xi, xj)
bgi = calc_gi(xib, xj)
print(np.linalg.norm(xib-xj))
print("calculated symfunc:", agi)
print("reference symfunc:", h_symfuncs[24])
print("numerical gradient: ", (bgi - agi)/s)
print(hgrad[24,0])


1.000001
calculated symfunc: 0.44041252110981277
reference symfunc: 0.4404081106185913
numerical gradient:  -3.4750755017465274
[-3.47508979  0.          0.        ]
